Implementation of Topic Model for NLP using LDA
following tutorial of Susan Li at https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [26]:
import sys
!{sys.executable} -m pip install gensim

    100% |████████████████████████████████| 24.7MB 24kB/s  eta 0:00:01    75% |████████████████████████        | 18.5MB 13.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 2.3MB/s ta 0:00:011
    100% |████████████████████████████████| 133kB 3.7MB/s eta 0:00:01
    100% |████████████████████████████████| 5.4MB 87kB/s  eta 0:00:01    60% |███████████████████▎            | 3.3MB 17.6MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 1.9MB/s ta 0:00:011
    100% |████████████████████████████████| 122kB 4.0MB/s eta 0:00:01
  Running setup.py bdist_wheel for smart-open ... done
  Stored in directory: /Users/Weiwei/Library/Caches/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built smart-open
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [50]:
#importing all needed libraries
import pandas as pd
#import gensim
#from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS as STOPWORDS
from nltk.stem import WordNetLemmatizer as wnLemmatizer
#from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
#import nltk
#nltk.download('wordnet')

In [7]:
# if running this implementation with shrinked data
# this cell is not needed
#data = pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)
#print(data.shape)
#data = data[:500000]
#data.to_csv('abcnews-short.csv')

(1103663, 2)


In [62]:
data = pd.read_csv('abcnews-short.csv',error_bad_lines=False)
documents = data[['headline_text']]
documents.columns = ['headline']
print(len(documents))
print(documents[:5])

500000
                                            headline
0  aba decides against community broadcasting lic...
1     act fire witnesses must be aware of defamation
2     a g calls for infrastructure protection summit
3           air nz staff in aust strike for pay rise
4      air nz strike to affect australian travellers


In [63]:
#function to lemmatize and stemming words
def lem_stem(text):
    return nltk.stem.WordNetLemmatizer().lemmatize(text,pos='v')

#function of preprocessing
#filter stopwords and words less than 3 characters, then lem&stem
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text,min_len=4):
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(lem_stem(token))
    return result

In [64]:
doc_sample = documents.iat[4310,0]
print('original document: ',doc_sample)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document:  rain helps dampen bushfires


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [65]:
processed_docs = documents['headline'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline, dtype: object

In [66]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 community
2 decide
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


In [68]:
type(dictionary)

gensim.corpora.dictionary.Dictionary

In [69]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [70]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(bow_corpus[4310])
#and translate back to word with the dictionary
for i in range(len(bow_corpus[4310])):
    print(dictionary[bow_corpus[4310][i][0]])

[(110, 1), (480, 1), (946, 1), (4132, 1)]
help
rain
bushfires
dampen


In [58]:
#writing my own function to preprocess text
def preProcess(originalString,lowerCase=True,minLength=4):
    if lowerCase:
        originalString = originalString.lower()
    words = []
    for word in originalString.split(' '):
        if len(word) >= minLength:
            if word not in STOPWORDS:
                words.append(wnLemmatizer().lemmatize(word,pos='v'))
    return words

print('testing function:\n',preProcess(doc_sample))

testing function:
 ['rain', 'help', 'dampen', 'bushfires']


In [59]:
tokenized_docs = documents['headline_text'].map(preProcess)
tokenized_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

In [ ]:
#function to create a dictionary